In [2]:
import os
import sys

src_path = r"D:\SEM 4\CS516\Yelp Fairness Review\scripts"
if src_path not in sys.path:
    sys.path.append(src_path)

from preprocessing import (
    load_businesses, flag_major_chains,
    load_reviews_for_businesses, merge_reviews_businesses,
    sample_reviews_per_city
)

In [ ]:

#parameters
business_path = r'../Yelp-JSON/yelp_dataset/yelp_academic_dataset_business.json'
review_path = r'../Yelp-JSON/yelp_dataset/yelp_academic_dataset_review.json'
target_cities = ['Las Vegas', 'Toronto', 'Phoenix', 'Charlotte', 'Edinburgh','Pittsburgh']
hospitality_keywords = ['Hotel', 'Resort', 'Motel', 'Inn', 'Lodge', 'Hostel', 'Spa', 'Restaurant']
major_chains = ['Marriott', 'Hilton', 'Hyatt', 'IHG', 'Holiday Inn', 'Sheraton', 'Ritz', 'Westin', 'DoubleTree']


In [ ]:
# Load and filter businesses
business_df = load_businesses(business_path, target_cities, hospitality_keywords, major_chains)
business_df = flag_major_chains(business_df, major_chains)

# Load reviews
valid_business_ids = set(business_df['business_id'])
review_df = load_reviews_for_businesses(review_path, valid_business_ids)

In [ ]:
# Merge
df = merge_reviews_businesses(review_df, business_df)

# Sample per city to ensure no imbalance in data
df_sampled = sample_reviews_per_city(df, city_col='city', n_per_city=5000, random_state=42)

# Save prepared data
df_sampled.to_csv('../Yelp-JSON/yelp_dataset/prepared_hospitality_reviews.csv', index=False)